# Tests Rapidos

In [ ]:
vector = MatrizRala.getVectorOne(10)
for i in range(10):
    vector[i,0] = i
# print(vector)
vectorNumpy = vector.toNumpy()
# ordenar de mayor a menor
argSort = np.argsort(vectorNumpy, axis=0)[::-1]# ordena de mayor a menor
vectorNumpy = np.sort(vectorNumpy, axis=0)[::-1]

# print(argSort)

vectorNumpy
vectorRala = MatrizRala.fromNumpy(vectorNumpy)
print(vectorRala)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matricesRalas import *
import itertools
papers = 'papers/papers.csv'
citas = 'papers/citas.csv'
citas2 = 'papers/citas2.csv'

W = MatrizRala.getW(papers,citas)

N: = 629814


In [ ]:
Wt = W.t()

## W D y Paralelismo

In [ ]:
from multiprocess import Manager,Pool,cpu_count
import sys
sys.setrecursionlimit(22000)  
def dividir_diccionario(dic, n):
    """Divide el diccionario en n partes aproximadamente iguales."""
    print(f"CANT KEYS: {len(dic.keys())}")
    it = iter(dic)
    tamaño = len(dic)
    tamaño_parte = tamaño // n + (tamaño % n > 0)  # Determina el tamaño de cada parte, ajustando para que sean aproximadamente iguales
    partes = []
    for i in range(n):
        # Toma una porción del tamaño calculado del iterador
        parte = {k: dic[k] for k in itertools.islice(it, tamaño_parte)}
        partes.append(parte)
    return partes

def procesar_fragmento_diccionario(shared_dict,fragmento):
    # Supongamos que queremos contar cuántos valores cumplen cierta condición
    contador = 0
    for key in fragmento.keys():
        contador = 0
        fila = fragmento[key]
        nodo = fila.raiz
        while nodo:
            if(nodo.valor[1] == 1):
                contador += 1
            nodo = nodo.siguiente
        # Agregar contador a la matriz: mat_D[key,key] = contador
        if(contador != 0):
            shared_dict[key] = 1/contador
    
    #     if cumple_condicion(value):  # Define tu propia condición
    #         contador += 1
    # return contador

def D_with_readCsvMultithread(W):
    n_cpu = cpu_count()
    m,n = W.shape
    
    chunks: list = dividir_diccionario(W.filas,n_cpu)
    D = MatrizRala(m,n)
    for chunk in chunks:
        print(len(chunk))
    with Manager() as manager:
        print("LLEGUE")
        shared_dict = manager.dict()
        with Pool(n_cpu) as pool:
            pool.starmap(procesar_fragmento_diccionario,[(shared_dict,chunks[i]) for i in range(len(chunks))])
        
        for key, value in shared_dict.items():
            D[key,key] = value
    return D

D = D_with_readCsvMultithread(W)



ModuleNotFoundError: No module named 'multiprocess'

In [ ]:
i = 629634
fila_i = D.filas[i]
print(fila_i)
len(D.filas.keys())

In [ ]:




top_10 = np.zeros(10)
top_10_index = np.zeros(10, dtype=float)

for i in range(W.shape[0]):
    v = D[i,i]
    if v > top_10[0]:  # Si v es mayor que el más pequeño en top_10
        top_10[0] = v
        top_10_index[0] = i
        sort_indices = np.argsort(top_10)  # Ordena y obtiene índices
        top_10 = top_10[sort_indices]  # Reordena top_10
        top_10_index = top_10_index[sort_indices]  # Reordena los índices de acuerdo a top_10

print(top_10,"\n",top_10_index)

In [ ]:
fila_max = W.filas[top_10_index[-1]]
contador = 0
nodo = fila_max.raiz
while nodo:
    contador += 1
    nodo = nodo.siguiente
print(contador)

In [ ]:
i = 629634 + 1
# fila_i = D.filas[i]
# print(fila_i)
# len(D.filas.keys())
i in W.filas.keys()

In [ ]:
from matricesRalas import *

A = MatrizRala(3,3)
A[0,0] = 1
A[0,0]

## matmul

In [ ]:

    # def __matmul__( self, other ):
    #     # Esta funcion implementa el producto matricial (notado en Python con el operador "@" ) -> A @ B
        
    #     if self.shape[1] != other.shape[0]:
    #         raise ValueError("los tamaños no se pueden multiplicar")
        
    #     resultado = MatrizRala(self.shape[0],other.shape[1])
    #     cantFilas = len(self.filas)
    #     cont_i = 0
    #     #para todas las filas de self 
    #     for current_i in self.filas: #valor de m
    #         cont_i += 1
            
    #         #agarro el raiz de la fila 
    #         fila = self.filas[current_i]
    #         if fila.raiz:
    #             rootNode_self = fila.raiz
    #             for j in range(other.shape[1]):
    #                 currentNode = rootNode_self
    #                 suma = 0
    #                 while currentNode is not None:
                        
    #                     current_j = currentNode.valor[0]
    #                     suma += currentNode.valor[1] * other[current_j,j]
    #                     currentNode = currentNode.siguiente

    #                 resultado[current_i,j] = suma
    #         # else:
    #         #     resultado[current_i,j] = 0



In [ ]:
 def __copy__(self):
        resultado = MatrizRala(self.shape[0],self.shape[1])
        for i in self.filas:
            
            fila = self.filas[i]
            nodo = fila.raiz
            #si hay algo en la lista
            if nodo is not None:
                
                #creo una lista enlazada 
                resultado.filas[i] = ListaEnlazada.Nodo(nodo.valor, None)  
                #guardo la posicion del nodo nuevo
                nuevo_nodo = resultado.filas[i]
                #voy al proximo elemento de la lista que estoy copiando
                nodo_actual = nodo.siguiente
                while nodo_actual is not None:
                    #el push agrega el valor del sigueinte nodo en la lista al final de la lista 
                    nuevo_nodo.push(nodo_actual.valor)  
                    #voy al siguiente nodo que se acaba de guarda
                    nuevo_nodo = nuevo_nodo.siguiente
                    #acanzo uno mas en la lissta og 
                    nodo_actual = nodo_actual.siguiente
          
            # for j in range(self.shape[1]):
            #     resultado[i,j] = self[i,j]
                
        return resultado

In [ ]:
import csv 
from matricesRalas import *
import matplotlib.pyplot as plt

#TIEMPO: 
papers = 'papers/papers.csv'
citas = 'papers/citas.csv'


W = MatrizRala.getW(papers,citas)



N: = 629814


In [ ]:
def getW(paperPath:str,citasPath:str):
        #tengo que tener agarrar la primer columna entera de papers csv 
        #cada posicion me da el nuumero del paper = paper + 1
        #ahora tengo que linkear cada con citas csv para crear la matriz
        #pj cita a pi id1 cita a id2 => W{id2,id1} =  1
        #1. crear matriz rala con dimension numero max de citas.csv m y n el mismo numero
        #2. recorrer citas csv por cada row ponerle set item (id2,id1) v=1
        
        #despues seria sobre len(ids)
    N = 0 
    with open(paperPath,newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)
              
        N = sum(1 for row in reader)
        csvfile.close()


    print(f"N: = {N}")
    W = MatrizRala(N,N)

    with open(citasPath, newline='') as csvfile:
        reader = csv.reader(csvfile)
        next(reader) #skipping the header
            # print("HOLA")
        for row in reader:
                # print(row)
                # print("1")
                #ai existe el 0
            to_ = int(row[1])
            from_ = int(row[0])
                #print(f" from_ = {from_}, to_ = {to_}")
                # pass
            W[to_,from_] = 1
        csvfile.close()

                
    return W

            